In [1]:
# Find RL_Note path and append sys path
import os, sys
cwd = os.getcwd()
pos = cwd.find('RL_Note')
root_path = cwd[0:pos] + 'RL_Note'
sys.path.append(root_path)

In [2]:
# Refer from
#  https://pasus.tistory.com/138
#  https://horomary.hatenablog.com/entry/2020/06/26/003806
#  https://keras.io/examples/rl/ddpg_pendulum/
#
import gym
import sys
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Concatenate, concatenate, Lambda
import matplotlib.pyplot as plt
from pys.utils.ou_noise import OUActionNoise
from pys.utils.prioritized_memory import ProportionalPrioritizedMemory

In [3]:
class Actor(tf.keras.Model):
    def __init__(self, state_size, action_size, action_min, action_max):
        super(Actor, self).__init__()
        self.action_min = action_min
        self.action_max = action_max

        self.fc1 = Dense(64, activation='relu')
        self.fc2 = Dense(64, activation='relu')
        # self.fc3 = Dense(16, activation='relu')
        self.out= Dense(action_size, activation='tanh',kernel_initializer = tf.random_uniform_initializer(minval=-0.003, maxval=0.003)) # -1 ~ +1

    def call(self, x):
        x       = self.fc1(x)
        x       = self.fc2(x)
        # x       = self.fc3(x)
        action  = self.out(x)
        # return self.projected_to_action_space(action)
        a = Lambda(lambda x: x*self.action_max)(action)
        return a

class Critic(tf.keras.Model):
    def __init__(self, state_size, action_size):
        super(Critic, self).__init__()
        self.s1 = Dense(16, activation='relu')
        self.s2 = Dense(32, activation='relu')
        self.a1 = Dense(32, activation='relu')
        self.a2 = Dense(32, activation='relu')
        self.fc1= Dense(64, activation='relu')
        self.fc2= Dense(64, activation='relu')
        self.out= Dense(1,  activation='linear')

    def call(self,state_action):
        state  = state_action[0]
        action = state_action[1]
        s = self.s1(state)
        s = self.s2(s)
        a = self.a1(action)
        a = self.a2(a)
        c = concatenate([s,a],axis=-1)
        x = self.fc1(c)
        x = self.fc2(x)
        q = self.out(x)
        return q

In [4]:
class DDPGAgent:
    def __init__(self, state_size, action_size, action_min, action_max):
        self.state_size = state_size
        self.action_size= action_size
        self.action_min = action_min
        self.action_max = action_max

        # Hyper params for learning
        self.discount_factor = 0.99
        self.actor_learning_rate  = 0.001
        self.critic_learning_rate = 0.002
        self.tau = 0.005

        # Experience Replay
        self.batch_size = 64
        self.train_start = 2000
        self.memory_size = 50000
        # ER
        # self.memory = ReplayMemory(capacity=self.buffer_size)
        # PER
        self.memory = ProportionalPrioritizedMemory(capacity=self.memory_size)
        # HER
        
        self.critic_optimizer   = tf.keras.optimizers.Adam(lr=self.critic_learning_rate)
        self.actor_optimizer    = tf.keras.optimizers.Adam(lr=self.actor_learning_rate)

        self.critic         = Critic(self.state_size, self.action_size)
        self.target_critic  = Critic(self.state_size, self.action_size)
        self.actor          = Actor(self.state_size, self.action_size, self.action_min, self.action_max)
        self.target_actor   = Actor(self.state_size, self.action_size, self.action_min, self.action_max)
        self.actor.build(input_shape=(None, self.state_size))
        self.target_actor.build(input_shape=(None, self.state_size))
        state_in = Input((self.state_size,))
        action_in = Input((self.action_size,))
        self.actor(state_in)
        self.target_actor(state_in)
        self.critic([state_in, action_in])
        self.target_critic([state_in, action_in])
        # self.actor.summary()
        # self.critic.summary()
        
        self.hard_update_target_model()

        self.noise_std_dev = 0.2
        self.ou_noise = OUActionNoise(mean=np.zeros(1), std_deviation=float(self.noise_std_dev) * np.ones(1))
        self.show_media_info = False

    def remember(self, state, action, reward, next_state, done):
        state       = np.array(state,dtype=np.float32)
        action      = np.array(action,dtype=np.float32)
        reward      = np.array([reward],dtype=np.float32)
        done        = np.array([done],dtype=np.float32)
        next_state  = np.array(next_state,dtype=np.float32)
        transition  = (state, action, reward, next_state, done)
        self.memory.append(transition)

    def hard_update_target_model(self):
        self.target_actor.set_weights(self.actor.get_weights())
        self.target_critic.set_weights(self.critic.get_weights())

    def soft_update_target_model(self):
        tau = self.tau
        for (net, target_net) in zip(   self.actor.trainable_variables,
                                        self.target_actor.trainable_variables):
            target_net.assign(tau * net + (1.0 - tau) * target_net)
        for (net, target_net) in zip(   self.critic.trainable_variables,
                                        self.target_critic.trainable_variables):
            target_net.assign(tau * net + (1.0 - tau) * target_net)

    def get_action(self,state):
        state = tf.convert_to_tensor([state], dtype=tf.float32)
        action = self.actor(state)
        # Exploration and Exploitation
        action_from_net = action.numpy()[0]
        action_from_noise = self.ou_noise()
        return np.clip(action_from_net+action_from_noise,self.action_min,self.action_max)

    def train_model(self):
        # Train from Experience Replay
        # Training Condition - Memory Size
        if len(self.memory) < self.train_start:
            return 0.0,0.0
        # Sampling from the memory
        # ER
        # mini_batch = self.memory.sample(self.batch_size)
        # PER
        mini_batch, idxs, is_weights = self.memory.sample(self.batch_size)

        states      = tf.convert_to_tensor(np.array([sample[0] for sample in mini_batch]))
        actions     = tf.convert_to_tensor(np.array([sample[1] for sample in mini_batch]))
        rewards     = tf.convert_to_tensor(np.array([sample[2] for sample in mini_batch]))
        next_states = tf.convert_to_tensor(np.array([sample[3] for sample in mini_batch]))
        dones       = tf.convert_to_tensor(np.array([sample[4] for sample in mini_batch]))
        
        if self.show_media_info == False:
            self.show_media_info = True
            print('Start to train, check batch shapes')
            print('shape of states', np.shape(states),type(states))
            print('shape of actions', np.shape(actions),type(actions))
            print('shape of rewards', np.shape(rewards),type(rewards))
            print('shape of next_states', np.shape(next_states),type(next_states)) 
            print('shape of dones', np.shape(dones),type(dones))

        with tf.GradientTape() as tape:
            target_actions = self.target_actor(next_states,training=True)
            target_q = self.target_critic([next_states,target_actions],training=True)
            target_value = rewards + (1 - dones) * self.discount_factor * target_q
            q = self.critic([states, actions],training=True)
            td_error = target_value - q
            critic_loss = tf.math.reduce_mean(is_weights*tf.math.square(td_error))
            critic_loss_out = tf.math.reduce_mean(tf.math.square(td_error))
        critic_params = self.critic.trainable_variables
        critic_grads = tape.gradient(critic_loss, critic_params)
        self.critic_optimizer.apply_gradients(zip(critic_grads, critic_params))

        with tf.GradientTape() as tape:
            new_actions = self.actor(states,training=True)
            new_q = self.critic([states, new_actions],training=True)
            actor_loss = -tf.reduce_mean(new_q)
        actor_params = self.actor.trainable_variables
        actor_grads = tape.gradient(actor_loss, actor_params)
        self.actor_optimizer.apply_gradients(zip(actor_grads, actor_params))
        
        self.soft_update_target_model()
        sample_importance = td_error.numpy()
        for i in range(self.batch_size):
            self.memory.update(idxs[i],abs(sample_importance[i]))
        return critic_loss_out.numpy(), actor_loss.numpy()

    def save_model(self):
        self.actor.save_weights("./save_model/pendulum_ddpg_per_TF_actor", save_format="tf")
        self.critic.save_weights("./save_model/pendulum_ddpg_per_TF_critic", save_format="tf")
        return


In [5]:
%matplotlib tk

ENV_NAME = 'Pendulum-v0'
EPISODES = 200
END_SCORE = -200

if __name__ == "__main__":
    env = gym.make(ENV_NAME)
    state_size  = env.observation_space.shape[0]
    action_size = env.action_space.shape[0]
    action_min  = env.action_space.low[0]
    action_max  = env.action_space.high[0]

    agent = DDPGAgent(state_size, action_size, action_min, action_max)
    print('Env Name : ',ENV_NAME)
    print('States {0}, Actions {1}'.format(state_size, action_size))
    print('Action space {0:.2f} ~ {1:.2f}'.format(action_min, action_max))
    scores_avg, scores_raw, episodes, losses = [], [], [], []
    critic_mean, actor_mean = [], []
    score_avg = 0

    end = False
    show_media_info = True
    
    for e in range(EPISODES):
        done = False
        score = 0
        state = env.reset()
        critic_losses = []
        actor_losses = []
        while not done:
            # env.render()

            # Interact with env.
            action = agent.get_action(state)
            next_state, reward, done, info = env.step(action)
            agent.remember(state, action, reward, next_state, done)
            critic_loss, actor_loss = agent.train_model()
            state = next_state

            score += reward
            critic_losses.append(critic_loss)
            actor_losses.append(actor_loss)
            if show_media_info:
                print("State Shape : ", np.shape(state))
                print("Action Shape : ", np.shape(action))
                print("Reward Shape : ", np.shape(reward))
                print("done Shape : ", np.shape(done))
                show_media_info = False
            if done:
                score_avg = 0.9 * score_avg + 0.1 * score if score_avg != 0 else score
                print("episode: {0:3d} | score avg: {1:3.2f} | mem size {2:6d} |"
                    .format(e, score_avg, len(agent.memory)))

                episodes.append(e)
                scores_avg.append(score_avg)
                scores_raw.append(score)
                critic_mean.append(np.mean(critic_losses))
                actor_mean.append(np.mean(actor_losses))
                # View data
                plt.clf()
                plt.subplot(311)
                plt.plot(episodes, scores_avg, 'b')
                plt.plot(episodes, scores_raw, 'b', alpha=0.8, linewidth=0.5)
                plt.xlabel('episode'); plt.ylabel('average score'); plt.grid()
                plt.title('pendulum DDPG PER')
                plt.subplot(312)
                plt.plot(episodes, critic_mean, 'b.',markersize=3)
                plt.xlabel('episode'); plt.ylabel('critic loss'); plt.grid()
                plt.subplot(313)
                plt.plot(episodes, actor_mean, 'b.',markersize=3)
                plt.xlabel('episode'); plt.ylabel('actor loss'); plt.grid()
                plt.savefig("./result/pendulum_ddpg_per_TF.png")

                # 이동 평균이 0 이상일 때 종료
                if score_avg > END_SCORE:
                    agent.save_model()
                    end = True
                    break
        if end == True:
            env.close()
            np.save('./save_model/data/pendulum_ddpg_per_TF_epi',  episodes)
            np.save('./save_model/data/pendulum_ddpg_per_TF_scores_avg',scores_avg)
            np.save('./save_model/data/pendulum_ddpg_per_TF_scores_raw',scores_raw)
            np.save('./save_model/data/pendulum_ddpg_per_TF_critic_mean',critic_mean)
            np.save('./save_model/data/pendulum_ddpg_per_TF_actor_mean',actor_mean)
            print("End")
            break

Env Name :  Pendulum-v0
States 3, Actions 1
Action space -2.00 ~ 2.00
State Shape :  (3,)
Action Shape :  (1,)
Reward Shape :  ()
done Shape :  ()
episode:   0 | score avg: -1293.69 | mem size    200 |
episode:   1 | score avg: -1283.08 | mem size    400 |
episode:   2 | score avg: -1272.39 | mem size    600 |
episode:   3 | score avg: -1249.49 | mem size    800 |
episode:   4 | score avg: -1252.98 | mem size   1000 |
episode:   5 | score avg: -1245.81 | mem size   1200 |
episode:   6 | score avg: -1298.87 | mem size   1400 |
episode:   7 | score avg: -1268.62 | mem size   1600 |
episode:   8 | score avg: -1310.58 | mem size   1800 |
Start to train, check batch shapes
shape of states (64, 3) <class 'tensorflow.python.framework.ops.EagerTensor'>
shape of actions (64, 1) <class 'tensorflow.python.framework.ops.EagerTensor'>
shape of rewards (64, 1) <class 'tensorflow.python.framework.ops.EagerTensor'>
shape of next_states (64, 3) <class 'tensorflow.python.framework.ops.EagerTensor'>
shap

In [ ]:
env.close()